In [3]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import keras_nlp

2024-02-20 20:21:18.437575: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-20 20:21:18.437684: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-20 20:21:18.569446: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using TensorFlow backend


In [4]:
FILE_DIR = '/kaggle/input/contradictory-my-dear-watson/'
train_df = pd.read_csv(FILE_DIR + 'train.csv')
test_df = pd.read_csv(FILE_DIR + 'test.csv')
sample_submission_df = pd.read_csv(FILE_DIR + 'sample_submission.csv')

In [5]:
train_df.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [6]:
train_df.shape

(12120, 6)

In [7]:
# Check if the dataset is balanced
train_df['label'].value_counts()

label
0    4176
2    4064
1    3880
Name: count, dtype: int64

In [8]:
# What percent of the dataset is each language?
pd.set_option('display.float_format', lambda x: '%.2f' % x)
train_df['language'].value_counts() / train_df.shape[0] * 100

language
English      56.68
Chinese       3.39
Arabic        3.31
French        3.22
Swahili       3.18
Urdu          3.14
Vietnamese    3.13
Russian       3.10
Hindi         3.09
Greek         3.07
Thai          3.06
Spanish       3.02
Turkish       2.90
German        2.90
Bulgarian     2.82
Name: count, dtype: float64

In [9]:
print(f"Average premise length: {train_df['premise'].apply(lambda x: len(x)).mean():.2f}")
print(f"Average hypothesis length: {train_df['hypothesis'].apply(lambda x: len(x)).mean():.2f}")

Average premise length: 107.37
Average hypothesis length: 53.89


In [10]:
# View some sample observations of the data
print(f"Premise: {train_df['premise'][478]}")
print(f"Hypothesis: {train_df['hypothesis'][478]}")
print(f"Label: {train_df['label'][478]}\n")

print(f"Premise: {train_df['premise'][420]}")
print(f"Hypothesis: {train_df['hypothesis'][420]}")
print(f"Label: {train_df['label'][420]}\n")

print(f"Premise: {train_df['premise'][469]}")
print(f"Hypothesis: {train_df['hypothesis'][469]}")
print(f"Label: {train_df['label'][469]}")

Premise: That's it. The girl looked at him, then passed her hand across her forehead.
Hypothesis: The girl touched her forehead.
Label: 0

Premise: You've got the keys still, haven't you, Poirot? I asked, as we reached the door of the locked room. 
Hypothesis: I had the keys in my pocket.
Label: 1

Premise: yeah TI people yeah and so i just figured no it's just this area you know
Hypothesis: No, I figured is was all areas.
Label: 2


In [11]:
VALIDATION_PROP = 0.15
NUM_TRAIN_OBS = int(train_df.shape[0]*(1-VALIDATION_PROP))
BATCH_SIZE = 16

In [12]:
def split_labels(data, label):
    return (data[0], data[1]), label

tf_training_dataset = (tf.data.Dataset.from_tensor_slices(
        (train_df[['premise','hypothesis']].values, train_df['label'])))

train_dataset = tf_training_dataset.take(NUM_TRAIN_OBS)
val_dataset = tf_training_dataset.skip(NUM_TRAIN_OBS)

In [13]:
# Inspect data formatting
sample = train_dataset.batch(4).take(1).get_single_element()
sample

(<tf.Tensor: shape=(4, 2), dtype=string, numpy=
 array([[b'and these comments were considered in formulating the interim rules.',
         b'The rules developed in the interim were put together with these comments in mind.'],
        [b'These are issues that we wrestle with in practice groups of law firms, she said. ',
         b'Practice groups are not permitted to work on these issues.'],
        [b"Des petites choses comme celles-l\xc3\xa0 font une diff\xc3\xa9rence \xc3\xa9norme dans ce que j'essaye de faire.",
         b"J'essayais d'accomplir quelque chose."],
        [b"you know they can't really defend themselves like somebody grown uh say my age you know yeah",
         b"They can't defend themselves because of their age."]],
       dtype=object)>,
 <tf.Tensor: shape=(4,), dtype=int64, numpy=array([0, 2, 0, 0])>)

In [14]:
train_preprocessed = train_dataset.map(split_labels, num_parallel_calls=tf.data.AUTOTUNE).batch(
    BATCH_SIZE, drop_remainder=True).cache().prefetch(tf.data.AUTOTUNE)

val_preprocessed = val_dataset.map(split_labels, num_parallel_calls=tf.data.AUTOTUNE).batch(
    BATCH_SIZE, drop_remainder=True).cache().prefetch(tf.data.AUTOTUNE)

## **Establishing the baseline with BERT**

### Tiny BERT

In [13]:
bert_classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased", num_classes=3)

bert_classifier.fit(train_preprocessed, validation_data=val_preprocessed, epochs=1)

Attaching 'config.json' from model 'keras/bert/keras/bert_tiny_en_uncased/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/bert/keras/bert_tiny_en_uncased/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/bert/keras/bert_tiny_en_uncased/2' to your Kaggle notebook...
/opt/conda/lib/python3.10/site-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/opt/conda/lib/python3.10/site-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
Attaching 'tokenizer.json' from model 'keras/bert/keras/bert_tiny_en_unca

### Tiny BERT with a higher learning rate

In [15]:
bert_classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased", num_classes=3)

bert_classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(1e-4),
    metrics=["accuracy"],)

bert_classifier.fit(train_preprocessed, validation_data=val_preprocessed, epochs=1)

Attaching 'config.json' from model 'keras/bert/keras/bert_tiny_en_uncased/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/bert/keras/bert_tiny_en_uncased/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/bert/keras/bert_tiny_en_uncased/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/bert/keras/bert_tiny_en_uncased/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.txt' from model 'keras/bert/keras/bert_tiny_en_uncased/2' to your Kaggle notebook...


643/643 [==============================] - 66s 88ms/step - loss: 1.0816 - accuracy: 0.3902 - val_loss: 1.0433 - val_accuracy: 0.4436


### Mulit-lingual BERT

In [17]:
bert_classifier = keras_nlp.models.BertClassifier.from_preset(
"bert_base_multi", num_classes=3)

bert_classifier.fit(train_preprocessed, validation_data=val_preprocessed, epochs=1)

Attaching 'config.json' from model 'keras/bert/keras/bert_base_multi/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/bert/keras/bert_base_multi/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/bert/keras/bert_base_multi/2' to your Kaggle notebook...
/opt/conda/lib/python3.10/site-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/opt/conda/lib/python3.10/site-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
Attaching 'tokenizer.json' from model 'keras/bert/keras/bert_base_multi/2' to your Kaggl

643/643 [==============================] - 651s 938ms/step - loss: 0.9729 - sparse_categorical_accuracy: 0.5284 - val_loss: 0.8934 - val_sparse_categorical_accuracy: 0.5841


### RoBERTa

In [16]:
roberta_classifier = keras_nlp.models.RobertaClassifier.from_preset(
    "roberta_base_en", num_classes=3)

roberta_classifier.fit(train_preprocessed, validation_data=val_preprocessed, epochs=1)

Attaching 'config.json' from model 'keras/roberta/keras/roberta_base_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/roberta/keras/roberta_base_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/roberta/keras/roberta_base_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/roberta/keras/roberta_base_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/merges.txt' from model 'keras/roberta/keras/roberta_base_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.json' from model 'keras/roberta/keras/roberta_base_en/2' to your Kaggle notebook...
W0000 00:00:1708456755.841869     101 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


643/643 [==============================] - 610s 873ms/step - loss: 0.9915 - sparse_categorical_accuracy: 0.4601 - val_loss: 0.8085 - val_sparse_categorical_accuracy: 0.6056


### DistilBERT-multi

In [20]:
distil_bert_classifier = keras_nlp.models.DistilBertClassifier.from_preset(
    "distil_bert_base_multi", num_classes=3)

distil_bert_classifier.fit(train_preprocessed, validation_data=val_preprocessed, epochs=1)

Attaching 'config.json' from model 'keras/distil_bert/keras/distil_bert_base_multi/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/distil_bert/keras/distil_bert_base_multi/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/distil_bert/keras/distil_bert_base_multi/2' to your Kaggle notebook...
/opt/conda/lib/python3.10/site-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/opt/conda/lib/python3.10/site-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
Attaching 'tokenizer.json' from model 'keras/d

643/643 [==============================] - 325s 465ms/step - loss: 1.0254 - sparse_categorical_accuracy: 0.4742 - val_loss: 0.9540 - val_sparse_categorical_accuracy: 0.5476


## XLM-RoBERTa-multi

In [16]:
xlm_roberta_classifier = keras_nlp.models.XLMRobertaClassifier.from_preset(
    "xlm_roberta_base_multi", num_classes=3)

xlm_roberta_classifier.fit(train_preprocessed, validation_data=val_preprocessed, epochs=1)

Attaching 'config.json' from model 'keras/xlm_roberta/keras/xlm_roberta_base_multi/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/xlm_roberta/keras/xlm_roberta_base_multi/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/xlm_roberta/keras/xlm_roberta_base_multi/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/xlm_roberta/keras/xlm_roberta_base_multi/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/xlm_roberta/keras/xlm_roberta_base_multi/2' to your Kaggle notebook...


Epoch 1/3


W0000 00:00:1708461223.320431     101 hlo_rematerialization.cc:2946] Can't reduce memory use below 9.13GiB (9801091373 bytes) by rematerialization; only reduced to 9.87GiB (10595412652 bytes), down from 14.99GiB (16094195564 bytes) originally
W0000 00:00:1708461235.747701     101 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


643/643 [==============================] - ETA: 0s - loss: 1.1059 - sparse_categorical_accuracy: 0.3387

W0000 00:00:1708461779.514837     101 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


643/643 [==============================] - 617s 888ms/step - loss: 1.1059 - sparse_categorical_accuracy: 0.3387 - val_loss: 1.0987 - val_sparse_categorical_accuracy: 0.3457
Epoch 2/3
643/643 [==============================] - 597s 928ms/step - loss: 1.1037 - sparse_categorical_accuracy: 0.3321 - val_loss: 1.0992 - val_sparse_categorical_accuracy: 0.3457
Epoch 3/3
643/643 [==============================] - 567s 882ms/step - loss: 1.1036 - sparse_categorical_accuracy: 0.3283 - val_loss: 1.0989 - val_sparse_categorical_accuracy: 0.3053
